In [34]:
import os
import requests
import json
import re
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()

True

In [ ]:
def obtener_api_url(clave):
    # """Obtiene la URL de la API del archivo de configuración."""
    try:
        with open("api_config.txt", "r") as f:
            for linea in f:
                if clave in linea:
                    return linea.split("=")[1].strip()
    except FileNotFoundError:
        print("Archivo de configuración no encontrado.")
        return None

def obtener_informacion_usuario(credenciales):
    """Obtiene la información del usuario de la API de inicio de sesión."""
    url = obtener_api_url("login_api")
    try:
        response = requests.post(url, json=credenciales)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error al obtener la información del usuario: {e}")
        return None

def obtener_categorias_faq():
    """Obtiene las categorías de preguntas frecuentes de la API."""
    url = obtener_api_url("faq_api")
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        categorias = [item["nombre"] for item in data["values"]]
        return categorias
    except requests.exceptions.RequestException as e:
        print(f"Error al obtener las categorías de la API: {e}")
        return None

def analizar_consulta_login(consulta):
    """Analiza la consulta del usuario para solicitudes de inicio de sesión y extrae las credenciales."""
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash-exp",
        google_api_key=os.getenv("GENAI_API_KEY"),
        temperature=0.3
    )

    prompt = f"""
    Determina si la siguiente consulta es una solicitud de inicio de sesión. Si es así, extrae las credenciales como un diccionario JSON.

    Consulta: "{consulta}"

    Si la consulta es una solicitud de inicio de sesión, responde con el diccionario JSON de las credenciales. De lo contrario, responde "no".
    """

    respuesta = llm.invoke(prompt).content.strip()

    # Extraer el JSON puro de la respuesta
    json_match = re.search(r'\{.*\}', respuesta, re.DOTALL)
    if json_match:
        json_str = json_match.group(0)
        try:
            credenciales = json.loads(json_str)
            return credenciales
        except json.JSONDecodeError:
            print("Error al analizar el JSON extraído.")
            return None
    else:
        return None

def analizar_consulta_faq(consulta):
    """Analiza la consulta del usuario para preguntas frecuentes."""
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash-exp",
        google_api_key=os.getenv("GENAI_API_KEY"),
        temperature=0.3
    )

    prompt = f"""
    Determina si la siguiente consulta es una pregunta sobre las categorías de preguntas frecuentes:
    "{consulta}"

    Responde "sí" o "no".
    """

    respuesta = llm.invoke(prompt).content.lower()
    return "sí" in respuesta

# query = """cuales son las categorias de las faq-topics"""



credenciales = analizar_consulta_login(query)

if credenciales:
    informacion_usuario = obtener_informacion_usuario(credenciales)
    if informacion_usuario:
        print(informacion_usuario)
    else:
        print("No se pudo obtener la información del usuario.")
elif analizar_consulta_faq(query):
    categorias = obtener_categorias_faq()
    if categorias:
        respuesta = "Las categorías de preguntas frecuentes son: " + ", ".join(categorias)
        print(respuesta)
    else:
        print("No se pudo obtener la información de las categorías.")
else:
    print("La consulta no es una solicitud de inicio de sesión o pregunta de categorías válida.")

{'oauth2': {'access_token': 'JaxpXbpy0IkqkJX5SbGNVmmQLssKA3', 'expires_in': 36000, 'token_type': 'Bearer', 'scope': 'read write', 'refresh_token': 'Vj5VOdJWBMpL73OYgGZqd7JMbkVF7C'}, 'id': 8019, 'user': {'id': 8875, 'username': 'katherine.aguirreor@gmail.com', 'is_active': True, 'is_staff': True, 'user_permissions': [], 'groups': [10]}, 'app_token': '', 'tipo_identificacion': 'CODIGO', 'identificacion': 'katherine.aguirreor@gmail.com', 'municipio': 1, 'territorial': 8, 'grupo_piraguero': None, 'zona': 'ND', 'nombres': '', 'apellidos': '', 'genero': 'ND', 'ocupacion': '', 'rango_edad': 'INDETERMINADO', 'fecha_nacimiento': None, 'grupo_poblacional': 'NINGUNO', 'tipo_actor': 'Sin especificar', 'nivel_academico': 'Estudiante', 'empresa': None, 'organizacion': '', 'email': 'katherine.aguirreor@gmail.com', 'telefono': '', 'foto': None, 'flags': {'id': 7028, 'lider': False, 'custodio': False, 'mide_lluvia': False, 'reporta_lluvia': False, 'gestor_limnigrafico': False, 'gestor_pluviografico': F